# Importing libraries

In [110]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import json 


In [133]:
driver = webdriver.Chrome()
driver.get("https://www.airbnb.fr/s/paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-05-01&monthly_length=3&monthly_end_date=2024-08-01&price_filter_input_type=0&channel=EXPLORE&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&price_filter_num_nights=6&checkin=2024-05-06&checkout=2024-05-12&adults=2&children=2&query=Paris%2C%20France&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=62139ace-d3f9-42a5-876f-3a810c907664&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjowLCJ2ZXJzaW9uIjoxfQ%3D%3D")

# Functions creation

In [3]:
def get_text(xpath):

    """
    This function takes the xpath of an element as an argument
    It returns the text of the specified element
    """
    
    return driver.find_element(By.XPATH, xpath).text

In [4]:
def click(xpath):

    """
    This function takes the xpath of an element
    It clicks on the element with the specified xpath
    It doesn't return anything
    """
    
    driver.find_element(By.XPATH, xpath).click()

In [5]:
def next_page():

    """
    This function doesn't take anything as an argument
    It finds the element to go to the next page, and then clicks on it 
    """
    
    driver.find_element(By.XPATH, "//a[@aria-label='Suivant']").click()

In [6]:
def switch():

    """"
    This function doesn't take anything as an argument
    It retrieves the identifiers of the windows opened by the browser controlled by the driver
    It then switchs to the last window
    The function doesn't return anything
    """

    window_handles = driver.window_handles

    driver.switch_to.window(window_handles[-1])

In [7]:
def close():

    """
    This function doesn't take anything as an argument
    It closes the window the driver is currently on
    It uses the switch function, to go back on the main window
    The function doesn't return anything
    """
    
    driver.close()
    switch()

In [8]:
def get_price():

    """
    This function doesn't take anything as an argument
    It finds the price element
    It then uses the get_text function to return the price as a string
    """

    return get_text('/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]')

In [9]:
def get_equipments():

    """
    This function doesn't take anything as an argument
    It returns all equipments that are pre
    It filters out elements containing the text "Indisponible"
    and returns the remaining text content of those elements
    """

    driver.find_elements(By.XPATH,"//*[contains(text(), 'Afficher les')]")[0].click()
    time.sleep(.3)
    equipments = driver.find_elements(By.XPATH, ("//div[@class='rten07p atm_lo_1tcgj5g atm_le_1tcgj5g atm_5j_t94yts dir dir-ltr']"))
    equipments = [i.text for i in equipments if 'Indisponible' not in i.text]
    return equipments


In [10]:
def get_offers():
    
    """
    This funtions doesn't take anything as an argument
    It returns a list of the offers on the main window as selenium elements
    """

    return driver.find_elements(By.XPATH, "//div[@class='lxq01kf atm_9s_1txwivl atm_am_kyuy1d atm_ar_d67k9l l1tup9az atm_1p4glcj_1bp4okc dir dir-ltr']")

In [11]:
def go_to(element):

    """
    This function takes an element as an argument
    It excutes javascript code scroll the page until the specified element is visible within the browser window
    This function doesn't return anything
    """

    driver.execute_script("arguments[0].scrollIntoView(true);", element)

In [12]:
def get_description():

    """
    This function takes an element as an argument
    It returns the description of the offer written by the offerer
    """

    classe = "d1isfkwk atm_vv_1jtmq4 dir dir-ltr"
    return driver.find_element(By.XPATH, (f"//div[@class='{classe}']")).text

In [44]:
def get_size():

    """
    This function doesn't take anything as an element
    It uses the get_description function and then searches in it the square meters notation and define the index where
    the notation has been found
    It then extracts a string near the index and extracts just the numbers to get the size of the appartment
    """

    description = get_description()
    size = ""
    string = ""

    if description.__contains__('m²') or description.__contains__('m2'):
        if description.__contains__('m²'):
            index = description.index("m²")
        else:
            index = description.index("m2")
        
        for i in range(6):
            string += description[index-3 + i]

        

        for i in string:
            if i.isdigit():
                size+=i
        size = int(size[:-1])
        
    else:
        size = 0
   
    
    return size

In [162]:
def get_info():

    """
    This function doesn't take anything as an element
    It returns the information about the offer such as the capacity of the appartment, the number of bedrooms,
    the number of beds and the number of bathrooms
    This function isn't used for the moment because it needs to corrected
    """
    
    tag = "o1kjrihn atm_c8_km0zk7 atm_g3_18khvle atm_fr_1m9t47k atm_h3_1y44olf atm_c8_2x1prs__oggzyc atm_g3_1jbyh58__oggzyc atm_fr_11a07z3__oggzyc dir dir-ltr"
    info = driver.find_element(By.XPATH, f"//div[@class='{tag}']").text.split("·")

    nb_of_travellers = int(info[0].split()[0])
    nb_of_bedrooms = int(info[1].split()[0])
    if info[2].__contains__('lits'):
        nb_of_beds = int(info[2].split()[0])
    else:
        nb_of_beds = 0
    # if info[-1].__contains__('de bain'):
    #     nb_of_bathrooms = int(info[-1].split()[0])
    # else : 
    #     nb_of_bathrooms = 0

    return nb_of_travellers, nb_of_bedrooms, nb_of_beds
    

In [128]:
def to_df(liste) :

    """
    This function takes a list as an argument
    The list given in argument should have the same format than the one from the scraping
    It returns a pandas dataframe from the list
    """

    dico = {}
    dico['prix'] = []
    dico['size'] = []
    dico['nb_of_travellers'] = []
    dico['nb_of_bedrooms'] = []
    dico['nb_of_beds'] = []
    dico['nb_of_bathrooms'] = []

    for i in liste:
        for j in i[2]:
            dico[j] = []

    for i in liste:
        prix = int(i[0].split()[0])
        size = i[1]
        equipements = i[2]
        dico['prix'].append(prix)
        dico['size'].append(size)
        for j in dico:
            if j != "prix" and j != "size":
                if j in equipements:
                    dico[j].append(1)
                else:
                    dico[j].append(0)

    df = pd.DataFrame(dico)

    return df

# SCRAPING


In [163]:
switch()

In [164]:
time.sleep(2)

liste = []
for j in range(15):
    
    offers = get_offers()
    for i in range(len(offers)):

        time.sleep(.5)
        go_to(offers[i])
        offers[i].click()
        time.sleep(.1)
        switch()
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]'))).click()
        except : 
            close()
            continue

        liste.append([])

        liste[-1].append(get_price())
        liste[-1].append(get_size())
        
        try:
            liste[-1].append(get_equipments())
        except:
            liste[-1].append([])
        close()
    if j!=14:
        next_page()
    time.sleep(2)


In [165]:
df = to_df(liste)

In [175]:
df

,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,Lave-linge,...,Caméras de surveillance extérieures présentes sur place\nLes caméras et alarmes sont situées à l'extérieur du logement pour prévenir de la sécurité du logement uniquement.,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,"TV avec Amazon Prime Video, Netflix, télévision par câble haut de gamme","Lit pour bébé : toujours dans le logement\nStandard (1,3 m de long sur 70 cm de large)",Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG
0,100,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,60,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,95,50,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,132,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,112,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,150,36,0,0,0,0,0,0,1,0,...,0,0,0,1,1,1,1,1,0,0
264,109,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
265,120,55,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,1,1
266,210,47,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [170]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as mse

y = df['prix']
X = df.drop('prix', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse(y_test, y_pred)**.5


38.470046067615314

In [171]:
y_test.mean()

114.62962962962963

In [172]:
mse(y_test, y_pred)**.5/y_test.mean()

0.33560298669648253